# 1511. Customer Order Frequency
[問題](https://leetcode.com/problems/customer-order-frequency/description/?envType=study-plan-v2&envId=premium-sql-50)

## 解答の方針
月ごと, customer_id毎の集計をして、該当するcustomer_idのシリーズを得てisin()で絞る。  
customersは最後にmergeする。

In [1]:
import pandas as pd

In [50]:
data = [[1, 'Winston', 'USA'], [2, 'Jonathan', 'Peru'], [3, 'Moustafa', 'Egypt']]
customers = pd.DataFrame(data, columns=['customer_id', 'name', 'country']).astype({'customer_id':'Int64', 'name':'object', 'country':'object'})
data = [[10, 'LC Phone', 300], [20, 'LC T-Shirt', 10], [30, 'LC Book', 45], [40, 'LC Keychain', 2]]
product = pd.DataFrame(data, columns=['product_id', 'description', 'price']).astype({'product_id':'Int64', 'description':'object', 'price':'Int64'})
data = [[1, 1, 10, '2020-06-10', 1], [2, 1, 20, '2020-07-01', 1], [3, 1, 30, '2020-07-08', 2], [4, 2, 10, '2020-06-15', 2], [5, 2, 40, '2020-07-01', 10], [6, 3, 20, '2020-06-24', 2], [7, 3, 30, '2020-06-25', 2], [9, 3, 30, '2020-05-08', 3]]
orders = pd.DataFrame(data, columns=['order_id', 'customer_id', 'product_id', 'order_date', 'quantity']).astype({'order_id':'Int64', 'customer_id':'Int64', 'product_id':'Int64', 'order_date':'datetime64[ns]', 'quantity':'Int64'})

In [51]:
customers

,customer_id,name,country
0,1,Winston,USA
1,2,Jonathan,Peru
2,3,Moustafa,Egypt


In [52]:
product

,product_id,description,price
0,10,LC Phone,300
1,20,LC T-Shirt,10
2,30,LC Book,45
3,40,LC Keychain,2


In [53]:
orders

,order_id,customer_id,product_id,order_date,quantity
0,1,1,10,2020-06-10,1
1,2,1,20,2020-07-01,1
2,3,1,30,2020-07-08,2
3,4,2,10,2020-06-15,2
4,5,2,40,2020-07-01,10
5,6,3,20,2020-06-24,2
6,7,3,30,2020-06-25,2
7,9,3,30,2020-05-08,3


In [54]:
def customer_order_frequency(customers: pd.DataFrame, product: pd.DataFrame, orders: pd.DataFrame) -> pd.DataFrame:
    orders = orders.merge(product, on="product_id", how="inner")
    orders["value"] = orders["quantity"] * orders["price"]
    grouped = orders.groupby([orders["order_date"].dt.to_period("M"), "customer_id"])["value"].sum().reset_index()
    grouped = grouped[grouped["value"] >= 100]
    june2020 = grouped[grouped["order_date"] == "2020-06"]["customer_id"]
    july2020 = grouped[grouped["order_date"] == "2020-07"]["customer_id"]
    result = grouped[grouped["customer_id"].isin(june2020) & grouped["customer_id"].isin(july2020)]
    result = result.merge(customers, on="customer_id")
    result = result.drop_duplicates(subset="customer_id")
    return result[["customer_id", "name"]]

In [55]:
customer_order_frequency(customers, product, orders)

,customer_id,name
0,1,Winston


## Test scripts

In [7]:
orders = orders.merge(product, on="product_id", how="left")

In [8]:
orders

,order_id,customer_id,product_id,order_date,quantity,description,price
0,1,1,10,2020-06-10,1,LC Phone,300
1,2,1,20,2020-07-01,1,LC T-Shirt,10
2,3,1,30,2020-07-08,2,LC Book,45
3,4,2,10,2020-06-15,2,LC Phone,300
4,5,2,40,2020-07-01,10,LC Keychain,2
5,6,3,20,2020-06-24,2,LC T-Shirt,10
6,7,3,30,2020-06-25,2,LC Book,45
7,9,3,30,2020-05-08,3,LC Book,45


In [9]:
orders["value"] = orders["quantity"] * orders["price"]

In [10]:
orders

,order_id,customer_id,product_id,order_date,quantity,description,price,value
0,1,1,10,2020-06-10,1,LC Phone,300,300
1,2,1,20,2020-07-01,1,LC T-Shirt,10,10
2,3,1,30,2020-07-08,2,LC Book,45,90
3,4,2,10,2020-06-15,2,LC Phone,300,600
4,5,2,40,2020-07-01,10,LC Keychain,2,20
5,6,3,20,2020-06-24,2,LC T-Shirt,10,20
6,7,3,30,2020-06-25,2,LC Book,45,90
7,9,3,30,2020-05-08,3,LC Book,45,135


In [20]:
grouped = orders.groupby([orders["order_date"].dt.to_period("M"), "customer_id" ])["value"].sum().reset_index()

In [25]:
grouped = grouped[grouped["value"] >= 100]

In [26]:
grouped

,order_date,customer_id,value
0,2020-05,3,135
1,2020-06,1,300
2,2020-06,2,600
3,2020-06,3,110
4,2020-07,1,100


In [27]:
june2020 = grouped[grouped["order_date"] == "2020-06"]["customer_id"]

In [29]:
july2020 = grouped[grouped["order_date"] == "2020-07"]["customer_id"]

In [32]:
june2020

1    1
2    2
3    3
Name: customer_id, dtype: Int64

In [38]:
answer = grouped[grouped["customer_id"].isin(july2020) & grouped["customer_id"].isin(june2020)]

In [43]:
answer = answer.drop_duplicates(subset="customer_id")

In [46]:
answer = answer.merge(customers, on="customer_id", how="inner")

In [47]:
answer[["customer_id", "name"]]

,customer_id,name
0,1,Winston
